# Библиотеки и функции

In [11]:
import pandas as pd
from graphlib.structures import Graph
from graphlib.algorithms import DFS_with_cc, BFS_search
from graphlib.tools import radius_approximate, diameter_approximate

# Загрузка данных

In [12]:
vk_dataset = pd.read_csv('datasets/vk.csv')
vk_dataset.head()

,u,v,t,h
0,3248374,11431799,27,6
1,3429130,4795235,13,8
2,105512,8910840,74,2
3,7900466,9739979,36,9
4,7296485,9838285,52,2


In [13]:
#G = Graph('vk')
ed = [(str(node1), str(node2)) for node1, node2 in zip(vk_dataset.u.tolist(), vk_dataset.v.tolist())]
print(len(ed))

17414510


In [14]:
G = Graph('vk', edges=ed)
print(G)

Graph <vk> with 3215720 nodes and 17414510 edges


# Выделение компонент связности, характеристики наибольшей компоненты

In [15]:
%%time

number, largest_index, components = DFS_with_cc(G, largest=True)
largest_component = G.subgraph(nodes=components[largest_index])
largest_component._name = 'the largest component::vk'
print(largest_component)

print(f'Доля вершин в наибольшей компоненте: {round(largest_component.nodes_count / G.nodes_count, 3)}')

Graph <subgraph::vk> with 3162217 nodes and 17378782 edges
Доля вершин в наибольшей компоненте: 0.983
Wall time: 22min 5s


# Метрические характеристики графа: диаметр, радиус, 90 процентиль (approximation)